In [37]:
import openai
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
from dotenv import load_dotenv

import plotly.express as px
import os
import pandas as pd
import numpy as np

In [17]:
load_dotenv()

True

In [18]:
openai.api_key = os.getenv("OPENAPI_KEY")

In [19]:
word_list = [
    "France",
    "Germany",
    "Aldi",
    "Lidl",
    "Cheese",
    "Porsche",
    "Cyberpunk",
    "Winnie the Poo",
    "John Cena",
    "Moon",
    "Programming",
    "REST API",
    "Feng Shui",
    "Qi",
    "Red",
    "Meth",
    "Methane",
    "Goodman",
    "Ellen Ripley",
    "Meat",
    "T800",
    "Stan Winston",
]

In [20]:
word_embeddings = []
for word in word_list:
    word_embeddings.append(get_embedding(word, engine="text-embedding-ada-002"))

In [21]:
# Make term and emebddings df
df = pd.DataFrame({"Word List": word_list, "Embeddings": word_embeddings})

In [22]:
df

,Word List,Embeddings
0,France,"[0.03075573965907097, -0.005854075308889151, -..."
1,Germany,"[0.019835034385323524, -0.026546243578195572, ..."
2,Aldi,"[-0.007395139895379543, -0.004052354488521814,..."
3,Lidl,"[-0.0035336303990334272, 0.0005827172426506877..."
4,Cheese,"[0.010652019642293453, 0.0009668035199865699, ..."
5,Porsche,"[0.00558151351287961, -0.0038178989198058844, ..."
6,Cyberpunk,"[0.0036292874719947577, -0.017004676163196564,..."
7,Winnie the Poo,"[0.0006167097599245608, -0.02209647372364998, ..."
8,John Cena,"[-0.013514889404177666, 0.004290861543267965, ..."
9,Moon,"[0.018877558410167694, -0.008514802902936935, ..."


In [65]:
def embed(term):
    return get_embedding(term, engine="text-embedding-ada-002")


def check_similarity(search, compare):
    search_embedding = embed(search)
    compare_embedding = embed(compare)
    return cosine_similarity(search_embedding, compare_embedding)


def search_reviews(df, search_term, pprint=True):
    embedding = embed(search_term)
    df['similarities'] = df["Embeddings"].apply(lambda x: cosine_similarity(x, embedding))
    res = df.sort_values('similarities', ascending=False)
    plotSimilarities(df, search_term)
    return res

In [95]:
res = search_reviews(df, "The deep see underwater expedition was a huge success. They found spongebob")

In [94]:
def plotSimilarities(df, search_term):
    fig = px.scatter(
        df, 
        x="similarities", 
        y=[0]*len(df["similarities"]), 
        hover_name="Word List",
        title="For Search Term:" + search_term    )
    #fig.update_layout(xaxis=dict(range=[0, 1]))
    fig.show()